In [1]:
import librosa
import pandas as pd
import torch.optim as optim
import numpy as np
import csv
from data_load import TextTransform
from difflib import SequenceMatcher
import os
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
#from decoder import GreedyDecoder
from torch.functional import F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import torch.nn as nn
import librosa
from librosa.core import stft, magphase
from glob import glob
from torch import autograd
import csv
from data_load import CodeSwitchDataset
import zipfile

/home/dexter/Desktop/MSR challenge/hparam.py:11: YAMLLoadWarning: calling yaml.load_all() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  for doc in docs:


In [2]:
def pad(wav, trans, lang):
    if lang == "Gujarati":
        max_len = 0
    elif lang == "Telugu":
        max_len = 529862
    elif lang == 'Tamil':
        max_len = 0
    else:
        raise Exception("Check Language")

    while len(wav) < max_len:
        diff = max_len - len(wav)
        ext = wav[:diff]
        wav = np.append(wav, wav[:diff])
        ratio = int(len(trans)*diff/len(wav))
        trans +=trans[:ratio]
    return wav, trans

def preprocess(data):
    #print(data)
    inputs = []
    labels = []
    input_lengths = []
    label_lengths = []
    
    for (wav, sr, trans, lang) in data:
        #wav, trans  = pad(wav, trans, lang)
        out = stft(wav, win_length=int(sr*0.02), hop_length=int(sr*0.01))
        out = np.transpose(out, axes=(1, 0))

        text_transform = TextTransform()
        trans = torch.Tensor(text_transform.text_to_int(trans.lower()))

        out = magphase(out)[0]
        out = torch.from_numpy(np.array([np.log(1 + x) for x in out]))
        #print(out.shape)
        inputs.append(out)
        labels.append(trans)
        input_lengths.append(out.shape[0])
        label_lengths.append(len(trans))
    inputs = nn.utils.rnn.pad_sequence(inputs, batch_first=True)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=0)
    #spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    return inputs, labels, input_lengths, label_lengths

In [4]:
def GreedyDecoder(output, labels, label_lengths, blank_label=0, collapse_repeated=False):
    arg_maxes = torch.argmax(output, dim=2)
    decodes = []
    text_transform = TextTransform()
    targets = []
    for i, args in enumerate(arg_maxes):
        decode = []
        targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
        for j, index in enumerate(args):
            if index != blank_label:
                if collapse_repeated and j != 0 and index == args[j -1]:
                    continue
                decode.append(index.item())
        decodes.append(text_transform.int_to_text(decode))
    return decodes, targets

In [5]:
train_dataset = CodeSwitchDataset(lang = 'Telugu', mode = "train")
validation_split = .2
shuffle_dataset = True
random_seed= 42
dataset_size = len(train_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


In [6]:
train_loader = DataLoader(train_dataset,
                          batch_size=8,
                          drop_last=True,
                          num_workers = 6,
                          sampler = train_sampler,
                         collate_fn = lambda x: preprocess(x))
test_loader = DataLoader(train_dataset,
                          batch_size=8,
                          drop_last=True,
                          num_workers = 6,
                          sampler = valid_sampler,
                         collate_fn = lambda x: preprocess(x))

In [7]:
device = torch.device('cuda')

In [15]:
class Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim = 29, num_layers = 4):
        super(Model, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.output_dim = output_dim
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        #self.drop = nn.Dropout(0.25)
        self.linear = nn.Linear(self.hidden_dim, self.output_dim)
        
    def forward(self, x):
        #print(type(self.lstm))
        #print(x.shape)
        lstm_out, hidden = self.lstm(x)
        #lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.linear(lstm_out)
        return out, hidden

In [16]:
torch.cuda.empty_cache()

In [10]:
data_len = len(train_loader.dataset)
pbar = tqdm(enumerate(train_loader), total=len(train_loader))
for batch_idx, (_data) in pbar:
    #bi, wav, label = batch_idx, wav, label
    wav, labels, input_lengths, label_lengths = _data
    print(wav.shape)

  0%|          | 1/822 [00:02<32:48,  2.40s/it]

torch.Size([8, 1278, 1025])
torch.Size([8, 1046, 1025])
torch.Size([8, 1040, 1025])
torch.Size([8, 1101, 1025])
torch.Size([8, 952, 1025])
torch.Size([8, 1121, 1025])


  1%|          | 9/822 [00:03<16:49,  1.24s/it]

torch.Size([8, 1068, 1025])
torch.Size([8, 907, 1025])
torch.Size([8, 1666, 1025])
torch.Size([8, 1157, 1025])
torch.Size([8, 1421, 1025])
torch.Size([8, 875, 1025])


  2%|▏         | 13/822 [00:05<12:58,  1.04it/s]

torch.Size([8, 993, 1025])


  2%|▏         | 14/822 [00:06<12:36,  1.07it/s]

torch.Size([8, 2288, 1025])
torch.Size([8, 856, 1025])
torch.Size([8, 1436, 1025])
torch.Size([8, 1392, 1025])
torch.Size([8, 804, 1025])


KeyboardInterrupt: 

In [17]:
def train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, writer):
    model.train()
    data_len = len(train_loader.dataset)
    total_loss=0
    train_loss=0
    avg_acc=0
    acc=[]
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for batch_idx, (_data) in pbar:
        #bi, wav, label = batch_idx, wav, label
        wav, labels, input_lengths, label_lengths = _data
        wav = wav.to(device)
        wav = wav.float()
        labels = labels.to(device)
        optimizer.zero_grad()
        #print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

        output, _ = model(wav)
        output = F.log_softmax(output, dim=1)
        output = output.transpose(0,1)
        #print(output.shape)
        loss = criterion(output, labels, input_lengths, label_lengths)
        #print(loss)
        total_loss+=loss
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        iter_meter.step()
        
        decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
        decoded_preds, decoded_targets = list(map(str.strip, decoded_preds)), list(map(str.strip, decoded_targets))
        for j in range(len(decoded_preds)):
            s = SequenceMatcher(None, decoded_targets[j], decoded_preds[j])
            acc.append(s.ratio())

        avg_acc = sum(acc)/len(acc)
        writer.add_scalar("train_accuracy", avg_acc, epoch)
        writer.add_scalar('train_loss', train_loss, epoch)
        writer.add_scalar('Loss', loss, epoch*len(train_loader)+1)
        writer.add_scalar('TLoss', total_loss, epoch*len(train_loader)+1)
        if batch_idx % 100 == 0 or batch_idx == data_len:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(wav), data_len,
                    100. * batch_idx / len(train_loader), loss.item()))
            print("Train Accuracy: {}, Test loss: {}".format(avg_acc, train_loss))
            
    print(decoded_preds)        
    if (epoch+1)%2 == 0:
        model.eval().cpu()
        ckpt_model_filename = "ckpt_epoch_" + str(epoch+1) + "_batch_id_" + str(batch_idx+1) + ".pth"
        ckpt_model_path = os.path.join("checkpoints/", ckpt_model_filename)
        torch.save(model.state_dict(), ckpt_model_path)
        model.to(device).train()

In [18]:
def test(model, device, test_loader, criterion, epoch, writer):
    model.eval()
    training_loss, train_acc = 0, 0
    eer, total_eer = 0, 0
    test_loss=0
    acc = []
    with torch.no_grad():
        for batch_idx, _data in enumerate(test_loader):
            inputs, labels, input_lengths, label_lengths = _data 
            inputs, labels = inputs.to(device), labels.to(device)

            output, _ = model(inputs)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=1)
            output = output.transpose(0, 1) # (time, batch, n_class)
            loss = criterion(output, labels, input_lengths, label_lengths)
            test_loss += loss.item() / len(test_loader)
            
            decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
            decoded_preds, decoded_targets = list(map(str.strip, decoded_preds)), list(map(str.strip, decoded_targets))
            for j in range(len(decoded_preds)):
                s = SequenceMatcher(None, decoded_targets[j], decoded_preds[j])
                acc.append(s.ratio())

            print(decoded_targets)
            print(decoded_preds)
            avg_acc = sum(acc)/len(acc)
            writer.add_scalar("test_accuracy", avg_acc, epoch)
            writer.add_scalar('test_loss', test_loss, epoch)
            print("Test Accuracy: {}, Test loss: {}".format(avg_acc, test_loss))

In [19]:
class IterMeter(object):
    """keeps track of total iterations"""
    def __init__(self):
        self.val = 0

    def step(self):
        self.val += 1

    def get(self):
        return self.val

In [20]:
model = Model(input_dim=1025,
              hidden_dim=512,
              batch_size=8,
              output_dim=3,
              num_layers=4)
#model.half()
model = model.to(device)
criterion = nn.CTCLoss().to(device)
epochs = 40
optimizer = optim.Adam(model.parameters(), 1e-3)

scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, 
                                            steps_per_epoch=int(len(train_loader)),
                                            epochs=40,
                                            anneal_strategy='linear')

In [13]:
for layer in model.modules():
    if isinstance(layer, nn.BatchNorm2d):
        layer.float()
writer = SummaryWriter('train_logs/')
iter_meter = IterMeter()
#checkpoint = torch.load("checkpoints/final_epoch_2_batch_id_18.model")
#model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch = checkpoint['epoch']
model.to(device)
model.train()
for epoch in range(1, epochs+1):
    train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, writer)
    test(model, device, test_loader, criterion, epoch, writer)
    
model.eval().cpu()
save_model_filename = "final_epoch_" + str(epoch + 1) + "_batch_id_" + str(batch_idx + 1) + ".model"
save_model_path = os.path.join("checkpoints", save_model_filename)
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, save_model_path)

print("\nDone, trained model saved at", save_model_path)

  0%|          | 1/822 [00:02<40:55,  2.99s/it]

Train Epoch: 1 [0/8225 (0%)]	Loss: 134.724304
Train Accuracy: 0.0, Test loss: 0


 12%|█▏        | 101/822 [00:52<05:30,  2.18it/s]

Train Epoch: 1 [800/8225 (12%)]	Loss: 124.767570
Train Accuracy: 0.01843025709165496, Test loss: 0


 24%|██▍       | 201/822 [01:40<04:51,  2.13it/s]

...

Train Epoch: 2 [0/8225 (0%)]	Loss: 111.547607
Train Accuracy: 0.017142857142857144, Test loss: 0


 12%|█▏        | 101/822 [00:51<06:37,  1.81it/s]

Train Epoch: 2 [800/8225 (12%)]	Loss: 119.096527
Train Accuracy: 0.008720346820260657, Test loss: 0


 21%|██        | 174/822 [01:25<04:50,  2.23it/s]

KeyboardInterrupt: 

In [54]:
checkpoint.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict'])

In [19]:
labels

22050

In [13]:
input_lengths

[618, 804, 443, 732, 502, 671, 505, 480]

In [21]:
inputs, labels, input_lengths, label_lengths = _data 
inputs, labels = inputs.to(device), labels.to(device)

output, _ = model(inputs)  # (batch, time, n_class)
output = F.log_softmax(output, dim=1)
output = output.transpose(0, 1) # (time, batch, n_class)
loss = criterion(output, labels, input_lengths, label_lengths)

decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, input_lengths, collapse_repeated=False)

In [26]:
label_lengths

[31, 41, 22, 37, 25, 34, 32, 24]

In [27]:
input_lengths

[618, 804, 443, 732, 502, 671, 505, 480]

In [28]:
decoded_preds[0], len(decoded_preds[0])

('tttttttttttt       t        ttt ttttt tttttttt  tttttt                 tttttt   ttttt  tttt tt                             tt  tttttttt         tt    ttt                 ttt  tttttt t ttttt                      ttttt         tttttt      t ttt         t             ttttt   t  t     ttttttt  tttt                     ttttttttt  tt t  tt t           ',
 348)

In [29]:
decoded_targets[0], len(decoded_targets[0])

('ssststtttttttttts  tttttttttttsssssssssss', 41)

In [30]:
labels[0]

tensor([0., 0., 0., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 1.,
        1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.], device='cuda:0')

In [31]:
output.shape

torch.Size([804, 8, 3])

In [26]:
inputs.shape

torch.Size([8, 896, 1025])

In [30]:
wav, sr = librosa.load('./Data/PartB_Gujarati/Train/Audio/000010173.wav')